In [39]:
# importing neccessary librararies 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#couldn't do it properly or convincingly because I don't know my matplotlib and seaborn lib were not working at all SORRY!

ImportError: DLL load failed while importing _cext: The specified module could not be found.

In [6]:
# load csv datasets using pandas library
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [11]:
print(customers.head())
print(products.head())
print(transactions.head())

# this is to check whether the loading of datasets was succesful or not


  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3       

Now we will have to merge the datasets carefully so that we can analyze all three together and perform EDA. Like in the above three datasets we will first merge transactions and customer datasets on the refefernce of "CUSTOMER ID" column and then merge products dataset with the resultant merged dataset of transactions and customers on the reference of "PRODUCT ID"

In [13]:
merged_data = pd.merge(transactions, customers, on="CustomerID", how="inner")
merged_data = pd.merge(merged_data, products, on="ProductID", how="inner")

In [19]:
print("missing values:", merged_data.isnull().sum())

# in this we have checked column wise total missing values 

missing values: TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price_x            0
CustomerName       0
Region             0
SignupDate         0
ProductName        0
Category           0
Price_y            0
dtype: int64


In [20]:
merged_data.describe()
# these are some statistics related to merged dataset that we formed like quartiles and deviations etc

,Quantity,TotalValue,Price_x,Price_y
count,1000.000000,1000.000000,1000.00000,1000.00000
mean,2.537000,689.995560,272.55407,272.55407
std,1.117981,493.144478,140.73639,140.73639
min,1.000000,16.080000,16.08000,16.08000
25%,2.000000,295.295000,147.95000,147.95000
50%,3.000000,588.880000,299.93000,299.93000
75%,4.000000,1011.660000,404.40000,404.40000
max,4.000000,1991.040000,497.76000,497.76000


Now we will start few data analysis related to business insights and how will we communivcate to the organization regarding  our insights which should inturn help them in gaining profits and identify more profit groups

In [21]:
# 1) let's first see in which region are we doing the best will have to focus more in that region rather than getting lighter in that
revenue = (merged_data.groupby('Region')['TotalValue'].sum().sort_values(ascending=False))
print(revenue)
# so according to this we get maximum transaction amount from south America region followed by Europe and least in Asia

Region
South America    219352.56
Europe           166254.63
North America    152313.40
Asia             152074.97
Name: TotalValue, dtype: float64


In [22]:
# 2) Let's see who are the most frequent customer on the basis of payment count so that we are able to keep them intact with us and give special attention to them so that we don't lose them as our customer
frequent_customers = (
    merged_data['CustomerName']
    .value_counts()
    .head(50)
)
print(frequent_customers)
# this is the list of 50 customers who are most frequent but is this all to conclude that they are the one who shall be given special attention
# No we shall see

CustomerName
Abigail Jones           11
William Adams           11
Paul Parsons            10
Matthew Johnson         10
Gerald Hines            10
David Li                10
Hunter Fuller            9
Nancy Walker             9
Misty Higgins            9
Belinda Garner           9
Juan Mcdaniel            9
Jessica Warren           8
Francisco Young          8
Laura Gilbert            8
Mark Cox                 8
Laura Bennett            8
Michael Atkinson         8
Mrs. Kimberly Wright     8
Kathleen Rodriguez       8
Wayne Stone              8
Bruce Rhodes             8
Ricky Gutierrez          8
Edwin Watson             8
Joseph Ortiz Jr.         8
Jennifer King            8
Rodney Eaton             8
Jason Yates              8
Kelsey Roberts           8
Robert Blanchard         8
Anna Ball                8
Jennifer Pena            8
David Davis              7
Michele Cooley           7
Lindsay Perez            7
Stacy Cook               7
Travis Campbell          7
Brian Parker   

In [27]:
# 3) Top customers according to total payment amounts
top_customers = merged_data.groupby('CustomerName')['TotalValue'].sum().sort_values(ascending=False).head(50)
print(top_customers)
#Now you can see that the few top customers in previous insight are not present here so we need to take both into consideration

CustomerName
Paul Parsons            10673.87
Bruce Rhodes             8040.39
Gerald Hines             7663.70
William Adams            7634.45
Aimee Taylor             7572.91
Anna Ball                7111.32
Mrs. Kimberly Wright     7073.28
Jennifer Pena            6819.57
Rodney Eaton             6715.72
Juan Mcdaniel            6708.10
Travis Campbell          6604.23
Laura Bennett            6579.10
Matthew Johnson          6210.53
Lindsey Deleon           6149.78
Michael Atkinson         6132.36
Benjamin Mcclure         6072.92
Brian Parker             6044.63
Michael Williams         6021.80
Lauren Buchanan          6000.56
Mark Cox                 5950.42
Misty Higgins            5848.97
Francisco Young          5808.04
Benjamin Anderson        5780.43
Tiffany Cain             5775.28
Wayne Stone              5771.27
Nicholas Ellis           5699.57
Beth Cardenas            5627.83
Cynthia Clayton          5610.25
Kelsey Roberts           5550.99
Abigail Jones            5487.

In [31]:
# here we will be checking the correlation between the two metrics we ggot before


transaction_count = merged_data.groupby('CustomerName')['TransactionID'].count().reset_index()
transaction_count.columns = ['CustomerName', 'TransactionCount']

# calculate total transaction amount for each customer
total_transaction_amount = merged_data.groupby('CustomerName')['TotalValue'].sum().reset_index()
total_transaction_amount.columns = ['CustomerName', 'TotalTransactionAmount']

# merge both metrics into a single DataFrame
customer_metrics = pd.merge(transaction_count, total_transaction_amount, on='CustomerName')

# sorting by transaction count and amount
customer_metrics_sorted = customer_metrics.sort_values(by='TransactionCount', ascending=False)

# Calculate correlation between transaction count and total transaction amount
correlation = customer_metrics['TransactionCount'].corr(customer_metrics['TotalTransactionAmount'])

# Display the results
print( customer_metrics_sorted)
print("\nCorrelation Between Transaction Count and Total Transaction Amount:\n", correlation)

# so this conveys that we can conclude from  any one of the above two metrics since correlation is strong approx 0.8
# this says strong linear realtionship between them as one increases other tends to increase


          CustomerName  TransactionCount  TotalTransactionAmount
196      William Adams                11                 7634.45
1        Abigail Jones                11                 5487.25
63        Gerald Hines                10                 7663.70
153       Paul Parsons                10                10673.87
47            David Li                10                 4271.61
..                 ...               ...                     ...
184          Tina Ford                 1                  137.54
143  Mr. Manuel Conway                 1                  922.41
74        James Murphy                 1                   82.36
46      David Gonzalez                 1                 1007.52
198     Zachary Turner                 1                  396.34

[199 rows x 3 columns]

Correlation Between Transaction Count and Total Transaction Amount:
 0.7947204249687639


In [33]:
# 4) Now lets see which is most frequently bought item 

top_product = merged_data['ProductName'].value_counts().head(5)
print(top_product)

# so most bought product is AcytiveWear Smartwatch closely followed by few more products

ProductName
ActiveWear Smartwatch    40
SoundWave Headphones     38
BookWorld Biography      30
ActiveWear Rug           29
SoundWave Cookbook       29
Name: count, dtype: int64


In [35]:
# 5) Now let's see what is which product category was contributing hoghest in the revenue

cat_revenue = merged_data.groupby('Category')['TotalValue'].sum().sort_values(ascending=False)
print(cat_revenue)

# this tells us otherwise means in most bought product it was dominated by electronics in the top but actually its the books first

Category
Books          192147.47
Electronics    180783.50
Clothing       166170.66
Home Decor     150893.93
Name: TotalValue, dtype: float64


In [36]:
correlation = merged_data[['Quantity', 'TotalValue']].corr()
print( correlation)

            Quantity  TotalValue
Quantity    1.000000    0.609972
TotalValue  0.609972    1.000000
